# Tutorial: Make an xarray Dataset CF-1.12 compliant

This notebook shows how to use `netcdf-cf-coercer` to:
- inspect CF compliance with `ds.cf.check_compliant()`
- apply safe fixes with `ds.cf.make_compliant()`

The object stays an `xarray.Dataset` throughout.


## 1. Setup


In [1]:
from __future__ import annotations

import numpy as np
import xarray as xr
from icecream import ic as print

import netcdf_cf_coercer  # registers ds.cf accessor


## 2. Build a dataset with missing CF metadata


In [2]:
ds = xr.Dataset(
    data_vars={
        "temperature": (("time", "lat", "lon"), np.random.rand(2, 3, 4)),
    },
    coords={
        "time": [0, 1],
        "lat": [10.0, 20.0, 30.0],
        "lon": [100.0, 110.0, 120.0, 130.0],
    },
)

ds


<xarray.Dataset> Size: 264B
Dimensions:      (time: 2, lat: 3, lon: 4)
Coordinates:
  * time         (time) int64 16B 0 1
  * lat          (lat) float64 24B 10.0 20.0 30.0
  * lon          (lon) float64 32B 100.0 110.0 120.0 130.0
Data variables:
    temperature  (time, lat, lon) float64 192B 0.8645 0.5462 ... 0.2428 0.7526

## 3. Check compliance before coercion

`check_compliant()` uses `cf-checker` when available. If `udunits2` is missing,
it falls back to heuristic checks and includes `checker_error` in the result.


In [3]:
issues_before = ds.cf.check_compliant()
print(issues_before)
print()


ic| issues_before: {'cf_version': 'CF-1.12',
                    'coordinates': {'lat': [{'message': '(3): No standard_name or long_name '
                                                        'attribute specified',
                                             'severity': 'WARN'},
                                            {'message': '(3.1): units attribute should be present',
                                             'severity': 'WARN'}],
                                    'lon': [{'message': '(3): No standard_name or long_name '
                                                        'attribute specified',
                                             'severity': 'WARN'},
                                            {'message': '(3.1): units attribute should be present',
                                             'severity': 'WARN'}],
                                    'time': [{'message': '(3): No standard_name or long_name '
                                                

## 4. Apply compliance fixes


In [4]:
fixed = ds.cf.make_compliant()

print(type(fixed))
print("Conventions:", fixed.attrs.get("Conventions"))
print("lat attrs:", fixed["lat"].attrs)
print("lon attrs:", fixed["lon"].attrs)
print("time attrs:", fixed["time"].attrs)

fixed


ic| type(fixed): <class 'xarray.core.dataset.Dataset'>
ic| 'Conventions:', fixed.attrs.get("Conventions"): 'CF-1.12'
ic| "lat attrs:": 'lat attrs:'
    fixed["lat"].attrs: {'axis': 'Y',
                         'long_name': 'latitude',
                         'standard_name': 'latitude',
                         'units': 'degrees_north'}
ic| "lon attrs:": 'lon attrs:'
    fixed["lon"].attrs: {'axis': 'X',
                         'long_name': 'longitude',
                         'standard_name': 'longitude',
                         'units': 'degrees_east'}
ic| "time attrs:": 'time attrs:'
    fixed["time"].attrs: {'axis': 'T', 'standard_name': 'time'}


<xarray.Dataset> Size: 264B
Dimensions:      (time: 2, lat: 3, lon: 4)
Coordinates:
  * time         (time) int64 16B 0 1
  * lat          (lat) float64 24B 10.0 20.0 30.0
  * lon          (lon) float64 32B 100.0 110.0 120.0 130.0
Data variables:
    temperature  (time, lat, lon) float64 192B 0.8645 0.5462 ... 0.2428 0.7526
Attributes:
    Conventions:  CF-1.12

## 5. Re-check after coercion


In [4]:
issues_after = fixed.cf.check_compliant()
print(issues_after)
print()

NameError: name 'fixed' is not defined

## 6. Other dimension patterns


In [14]:
# time-only
only_time = xr.Dataset(
    data_vars={"signal": (("time",), [1.0, 2.0, 3.0])},
    coords={"time": [0, 1, 2]},
)

# lat-lon only
only_space = xr.Dataset(
    data_vars={"field": (("lat", "lon"), np.ones((2, 2)))},
    coords={"lat": [0.0, 1.0], "lon": [10.0, 11.0]},
)

print("time-only conventions:", only_time.cf.make_compliant().attrs.get("Conventions"))
print("space-only conventions:", only_space.cf.make_compliant().attrs.get("Conventions"))


ic| "time-only conventions:": 'time-only conventions:'
    only_time.cf.make_compliant().attrs.get("Conventions"): 'CF-1.12'
ic| "space-only conventions:": 'space-only conventions:'
    only_space.cf.make_compliant().attrs.get("Conventions"): 'CF-1.12'


('space-only conventions:', 'CF-1.12')

## 7. Write to NetCDF (optional)

Once compliant, you can write normally:

```python
fixed.to_netcdf("example_cf.nc")
```
